In [1]:
import openpyxl
import re

In [2]:
def excel_to_list2d(file_path: str, sheet_name: str):
    wb = openpyxl.open(filename=file_path, data_only=True)
    ws = wb[sheet_name]
    list2d = [[cell.value for cell in row] for row in ws[1:ws.max_row]]
    
    return list2d

In [3]:
def list2d_to_new_excel(list2d: list, new_path: str):
    wb = openpyxl.Workbook()
    ws = wb.active

    for row_idx, row in enumerate(list2d):
        for col_idx, col in enumerate(row):
            val = list2d[row_idx][col_idx]
            ws.cell(
                row=row_idx+1,
                column=col_idx+1,
                value=val
            )

    wb.save(new_path)

In [4]:
def update_mst(mst):
    if mst in ["", None]:
        mst = None
    else:
        # Convert to upper-case string.
        mst = str(mst).upper()
        # Remove non-letter/non-number characters.
        mst = re.sub("[^A-Z0-9]", "", mst)
        # Add leading "0" character (if necessary).
        str_len = len(mst)
        if str_len in [9, 12] and mst[0] != "0":
            mst = re.sub("^", "0", mst)
        # Remove non-standard mst.
        str_len = len(mst)
        if str_len == 10:
            pass
        elif str_len == 13:
            mst = mst[:10] + "-" + mst[10:]
        else:
            mst = None
    
    return mst

In [5]:
unikey = {
    "a": [
        "a", "á", "à", "ả", "ã", "ạ",
        "ă", "ắ", "ằ", "ẳ", "ẵ", "ặ",
        "â", "ấ", "ầ", "ẩ", "ẫ", "ậ"
    ],
    "d": [
        "d", "đ"
    ],
    "e": [
        "e", "é", "è", "ẻ", "ẽ", "ẹ",
        "ê", "ế", "ề", "ể", "ễ", "ệ"
    ],
    "i": [
        "i", "í", "ì", "ỉ", "ĩ", "ị"
    ],
    "o": [
        "o", "ó", "ò", "ỏ", "õ", "ọ",
        "ô", "ố", "ồ", "ổ", "ỗ", "ộ",
        "ơ", "ớ", "ờ", "ở", "ỡ", "ợ"
    ],
    "u": [
        "u", "ú", "ù", "ủ", "ũ", "ụ",
        "ư", "ứ", "ừ", "ử", "ữ", "ự"
    ],
    "y": [
        "y", "ý", "ỳ", "ỷ", "ỹ", "ỵ"
    ]
}

In [6]:
uni_str = ""
for key, value in unikey.items():
    chars = []
    for char in value:
        chars.append(char)
    globals()["uni_{}".format(key)] = "".join(chars)
    uni_str += "".join(chars)

In [7]:
def regex(txt: str):
    chars = list(txt)
    idx_adj = 0
    for idx, char in enumerate(txt):
        if char in unikey.keys():
            chars.insert(idx+idx_adj, "[{")
            chars.insert(idx+idx_adj+2, "}]")
            idx_adj += 2
    txt = "".join(chars)
    txt = txt.format(
        a=uni_a, d=uni_d, e=uni_e, i=uni_i,
        o=uni_o, u=uni_u, y=uni_y
    )

    return txt

In [8]:
com_phrases = {
    "cn": ["chi nhanh"],
    "công ty": ["cong ty", "cty"],
    "dntn": ["doanh nghiep tu nhan", "doanh nghiep tn", "dn tu nhan", "dn *tn"],
    "tnhh": ["trach nhiem huu han", "tn hh"],
    "cp": ["co phan"],
    "mtv": ["mot thanh vien", "0*1 thanh vien", "0*1 *tv"],
    "vpđd": ["van phong dai dien", "vp dai dien", "vp *dd"],
    "vpcc": ["van phong cong chung", "vp cong chung", "vp cc"],
    "đđkd": ["dia diem kinh doanh", "dia diem kd", "dd kinh doanh", "dd *kd"],
    "htx": ["hop tac xa"],
    "hkd": ["ho kinh doanh", "ho kd", "hkd"],
    " đầu tư ": [" đt "],
    " sản xuất ": [" sx "],
    " xây dựng ": [" xd "],
    " thương mại ": [" tm "],
    " dịch vụ ": [" dv "],
    " du lịch ": [" dl "],
    " quảng cáo ": [" qc "],
    "tmdv": ["thuong mai [v&-]*a* *dich vu", "tm [v&-]*a* *dich vu", "thuong mai [v&-]*a* *dv"],
    "xnk": ["xuat [-]* *nhap khau", "x [-] n khau", "xn khau"],
    "bđs": ["bat dong san"]
}

In [9]:
def update_tendn(ten_dn):
    if ten_dn in ["", None]:
        ten_dn = None
    else:
        # Convert to lower-case string.
        ten_dn = str(ten_dn).lower()
        # Remove leading/trailing non-letter/non-number characters.
        ten_dn = re.sub("^[^0-9a-z{0}]+|[^0-9a-z{0}]+$".format(uni_str), "", ten_dn)
        # Replace U+2013 with U+002d, a more common dash symbol.
        ten_dn = re.sub("–", "-", ten_dn)
        # Remove some special characters.
        ten_dn = re.sub("[^0-9a-z{0}&.-]+".format(uni_str), " ", ten_dn)
        # Wrap special cases with whitespaces.
        ten_dn = re.sub("-", " - ", ten_dn)
        ten_dn = re.sub("&", " & ", ten_dn)
        # Compress adjacent whitespaces into one.
        ten_dn = re.sub(" +", " ", ten_dn)
        # Handle common phrases.
        for key, value in com_phrases.items():
            for phrase in value:
                ten_dn = re.sub(regex(phrase), key, ten_dn, 1)
        
        return ten_dn

In [10]:
table = excel_to_list2d("step_1.xlsx", "Sheet1")

In [11]:
mst_idx = 2
tendn_idx = 3
sbn_idx = 8

In [12]:
for row in table:
    row[mst_idx] = update_mst(row[mst_idx])
    row[tendn_idx] = update_tendn(row[tendn_idx])
    if row[sbn_idx] == "Sở Y tế":
        row[sbn_idx] = None

In [13]:
list2d_to_new_excel(table, "step_2.xlsx")